<a href="https://colab.research.google.com/github/phap-bot/dap301m/blob/main/Feature_m%C3%B4n_h%E1%BB%8Dc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tạo DataFrame dựa trên thông tin survey

In [ ]:
# Imports & Loading Data ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình hiển thị biểu đồ đẹp hơn
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Đọc dữ liệu (Thay đổi đường dẫn file nếu cần)
file_path = "/content/drive/MyDrive/Dự án/Export_Survey_1344_20251124_gui BM.xlsx"
try:
    df = pd.read_excel(file_path, sheet_name="diem")
    print("Đọc dữ liệu thành công!")
    display(df.head()) # Hiển thị 5 dòng đầu
    print(df.info())   # Kiểm tra kiểu dữ liệu
except FileNotFoundError:
    print(f"Không tìm thấy file tại: {file_path}. Vui lòng kiểm tra lại đường dẫn.")

Đọc dữ liệu thành công!


,ID SV,SemesterName,ClassName,SubjectCode,AverageMark,Status,StartDate,EndDate,Credits
0,173,Spring2023,SE1710_TF_CT,PRO192,6.9,Passed,03/01/2023,23/03/2023,3
1,173,Spring2024,JPD123.3W_TF_CT,JPD123,2.3,Fail,08/04/2024,26/04/2024,3
2,173,Summer2024,MAD101.3W_TF_CT,MAD101,5.7,Fail,05/08/2024,23/08/2024,3
3,173,Fall2024,SE1906_TF_CT,MAD101,0.0,Is AttendanceFail,04/09/2024,08/11/2024,3
4,173,Summer2024,JPD123.3W2_TF_CT,JPD123,4.3,Fail,05/08/2024,23/08/2024,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75964 entries, 0 to 75963
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID SV         75964 non-null  int64  
 1   SemesterName  75964 non-null  object 
 2   ClassName     75964 non-null  object 
 3   SubjectCode   75964 non-null  object 
 4   AverageMark   73204 non-null  float64
 5   Status        75964 non-null  object 
 6   StartDate     75964 non-null  object 
 7   EndDate       75964 non-null  object 
 8   Credits       75964 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 5.2+ MB
None


In [ ]:
pd.DataFrame(df.SubjectCode.unique())

,0
0,PRO192
1,JPD123
2,MAD101
3,ITE302c
4,ÐSA102
...,...
212,MSM201c
213,MCO201m
214,ENM301
215,ECO111


In [ ]:
df_subject = pd.DataFrame(df["SubjectCode"].str[:3])
df_subject

,SubjectCode
0,PRO
1,JPD
2,MAD
3,MAD
4,JPD
...,...
75959,PRO
75960,JPD
75961,OJT
75962,VOV


In [ ]:
pd.DataFrame(df_subject.SubjectCode.unique())

,0
0,PRO
1,JPD
2,MAD
3,ITE
4,ÐSA
...,...
148,MSM
149,MCO
150,ENM
151,ECO


In [ ]:
pass_rate_by_course = df.groupby("SubjectCode")["Status"].apply(lambda x: (x == "Passed").mean())
pass_rate_by_course

,Status
SubjectCode,
ACC101,0.920792
ADY201m,0.971751
AET102,1.000000
AFA201,1.000000
AID301c,1.000000
...,...
ÐBA102,0.977099
ÐNG102,0.957143
ÐSA102,0.938571


In [ ]:
import pandas as pd

# Giả sử df là DataFrame gốc
# df = pd.read_csv(...)

#---------------------------------------------------
# 1. Tỉ lệ rớt môn vì cháy
#---------------------------------------------------
fail_by_burst_rate = (
    df.groupby("SubjectCode")["Status"]
      .apply(lambda x: (x == "Is AttendanceFail").mean())
)
fail_by_burst_rate.name = "Tỉ lệ rớt do cháy môn"


#---------------------------------------------------
# 2. Tỉ lệ rớt do k đủ điểm
#---------------------------------------------------
fail_by_score_rate = (
    df.groupby("SubjectCode")["Status"]
      .apply(lambda x: ((x != "Passed") & (x != "Is AttendanceFail")).mean())
)
fail_by_score_rate.name = "Tỉ lệ rớt do không đủ điểm"

#---------------------------------------------------
# 3. Tỉ lệ điểm theo thang A-F
#---------------------------------------------------
def letter_grade(score):
    if score >= 9.0: return "Xuất Sắc"
    if score >= 8.0: return "Giỏi"
    if score >= 7.0: return "Khá Giỏi"
    if score >= 5.0: return "Khá"
    return "Rớt môn"

df["Grade"] = df["AverageMark"].apply(letter_grade)

grade_dist = pd.crosstab(df["SubjectCode"], df["Grade"], normalize="index")

#---------------------------------------------------
# 3.5. Trọng số (môn dễ lấy điểm hay khó)
#---------------------------------------------------
grade_weight = {
    "Xuất Sắc": 4,
    "Giỏi": 3,
    "Khá Giỏi": 2,
    "Khá": 1,
    "Rớt môn": 0
}

df["GradeWeight"] = df["Grade"].map(grade_weight)

ease_index = (
    df.groupby("SubjectCode")["GradeWeight"]
      .mean()
)
ease_index.name = "Mức độ dễ lấy điểm"

#---------------------------------------------------
# 4. Số lần mở lớp (dựa vào class_id)
#---------------------------------------------------
class_count = df.groupby("SubjectCode")["ClassName"].nunique()
class_count.name = "Số lượng lớp"

#---------------------------------------------------
# 5. Số lượng sinh viên mỗi lớp
#---------------------------------------------------
students_per_class = df.groupby(["SubjectCode", "ClassName"])["ID SV"].nunique()
avg_students = students_per_class.groupby("SubjectCode").sum()
avg_students.name = "Số lượng sv mỗi lớp"

#---------------------------------------------------
# 6. Số lượng sinh viên học lại
#---------------------------------------------------
df = df.sort_values(["ID SV", "SubjectCode"])

df["attempt"] = df.groupby(["ID SV", "SubjectCode"]).cumcount() + 1

all_courses = df["SubjectCode"].unique()
# Đếm số sinh viên theo course và attempt
retake_total = (
    df[df["attempt"] > 1]
      .groupby("SubjectCode")["ID SV"]
      .nunique()
      .reindex(all_courses, fill_value=0)    # đảm bảo môn không có học lại → 0
      .astype(int)
      .rename("Số lượng sv học lại")
)

#---------------------------------------------------
# 7. Gộp toàn bộ feature vào một bảng duy nhất
#---------------------------------------------------
course_features = pd.concat([
    fail_by_burst_rate,
    fail_by_score_rate,
    grade_dist,
    ease_index,
    class_count,
    avg_students,
    retake_total
], axis=1)

course_features.reset_index(inplace=True)
course_features


,SubjectCode,Tỉ lệ rớt do cháy môn,Tỉ lệ rớt do không đủ điểm,Giỏi,Khá,Khá Giỏi,Rớt môn,Xuất Sắc,Mức độ dễ lấy điểm,Số lượng lớp,Số lượng sv mỗi lớp,Số lượng sv học lại
0,ACC101,0.039604,0.039604,0.153465,0.455446,0.297030,0.074257,0.019802,1.589109,39,202,12
1,ADY201m,0.011299,0.016949,0.514124,0.107345,0.271186,0.016949,0.090395,2.553672,15,177,5
2,AET102,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1,1,0
3,AFA201,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1,1,0
4,AID301c,0.000000,0.000000,0.311111,0.200000,0.455556,0.000000,0.033333,2.177778,2,90,0
...,...,...,...,...,...,...,...,...,...,...,...,...
212,ÐBA102,0.022901,0.000000,0.419847,0.091603,0.251908,0.022901,0.213740,2.709924,25,131,2
213,ÐNG102,0.033333,0.009524,0.385714,0.161905,0.290476,0.042857,0.119048,2.376190,33,210,3
214,ÐSA102,0.050000,0.011429,0.308571,0.157143,0.321429,0.061429,0.151429,2.331429,83,700,11
215,ÐTB102,0.052174,0.000000,0.269565,0.182609,0.295652,0.052174,0.200000,2.382609,20,115,2


In [ ]:
row = course_features[course_features["Số lượng sv mỗi lớp"] == 1834]
row

,SubjectCode,Tỉ lệ rớt do cháy môn,Tỉ lệ rớt do không đủ điểm,Giỏi,Khá,Khá Giỏi,Rớt môn,Xuất Sắc,Mức độ dễ lấy điểm,Số lượng lớp,Số lượng sv mỗi lớp,Số lượng sv học lại
160,PRJ301,0.05764,0.241436,0.109842,0.406743,0.200653,0.258836,0.023926,1.233279,150,1834,370


In [ ]:
df_idx = df.set_index("SubjectCode")

result = (
    df_idx.loc["PRJ301"]
    .groupby("ClassName")
    .mean(numeric_only=True)
)
result

,ID SV,AverageMark,Credits,GradeWeight,attempt
ClassName,,,,,
3W_PRJ301_01,951.0,8.000000,3.0,3.0,1.000000
3W_PRJ301_02,895.0,5.450000,3.0,1.0,3.000000
3W_PRJ301_03,507.0,6.200000,3.0,1.0,2.000000
3W_PRJ301_04,375.0,6.850000,3.0,1.5,2.000000
3W_PRJ301_05,1140.0,5.500000,3.0,1.0,3.000000
...,...,...,...,...,...
SE1879,1095.0,6.733333,3.0,1.0,1.333333
SE18B02_TF_DN,758.0,7.300000,3.0,2.0,1.000000
SE18B04_TF_DN,1176.0,7.500000,3.0,2.0,1.000000


In [ ]:
course_features.describe()

,Tỉ lệ rớt do cháy môn,Tỉ lệ rớt do không đủ điểm,Giỏi,Khá,Khá Giỏi,Rớt môn,Xuất Sắc,Mức độ dễ lấy điểm,Số lượng lớp,Số lượng sv mỗi lớp,Số lượng sv học lại
count,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000,217.000000
mean,0.029129,0.071958,0.290330,0.207590,0.309967,0.088308,0.103805,2.113732,32.258065,349.764977,16.981567
std,0.092109,0.144217,0.291957,0.212293,0.281562,0.179348,0.200088,0.759773,53.932895,568.465580,46.457364
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.046875,0.000000,0.131429,0.000000,0.000000,1.592754,1.000000,2.000000,0.000000
50%,0.000000,0.012064,0.219512,0.174419,0.262463,0.016949,0.015082,2.039062,7.000000,63.000000,1.000000
75%,0.012658,0.070111,0.400000,0.328205,0.409091,0.071429,0.111111,2.675676,25.000000,273.000000,7.000000
max,0.750000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,203.000000,1834.000000,370.000000


# Tạo DataFrame encode one-hot thứ nhất

In [ ]:
import pandas as pd

# 1. Giữ Subjectcode
subject_code = course_features["SubjectCode"]

# 2. Dominant type từ các cột tỉ lệ
ratio_cols = ["Rớt môn", "Giỏi", "Khá", "Khá Giỏi", "Xuất Sắc"]

dominant = course_features[ratio_cols].idxmax(axis=1)

df_dom_onehot = pd.get_dummies(
    dominant,
    prefix="KQ_",
    dtype=int
)

# 3. Threshold-based one-hot
thresholds = {
    "Tỉ lệ rớt do cháy môn": [0.02, 0.04, 0.06, 0.08],
    "Tỉ lệ rớt do không đủ điểm": [0.02, 0.04, 0.06, 0.08],
    "Mức độ dễ lấy điểm": [1, 2, 3, 4],
    "Số lượng lớp": [1, 10, 20, 30, 40, 50],
    "Số lượng sv mỗi lớp": [1, 100, 200, 300, 400, 500],
    "Số lượng sv học lại": [10, 20, 30, 40, 50]
}

onehot_threshold = {}

for col, ths in thresholds.items():
    ths = sorted(ths)  # đảm bảo tăng dần

    for t in ths:
        onehot_threshold[f"{col} >= {t}"] = (
            course_features[col] >= t
        ).astype(int)

df_threshold = pd.DataFrame(
    onehot_threshold,
    index=course_features.index
)


# 4. Gộp tất cả thành DF feature cuối
course_features_onehot = pd.concat(
    [subject_code, df_dom_onehot, df_threshold],
    axis=1
)


In [ ]:
course_features_onehot

,SubjectCode,KQ__Giỏi,KQ__Khá,KQ__Khá Giỏi,KQ__Rớt môn,KQ__Xuất Sắc,Tỉ lệ rớt do cháy môn >= 0.02,Tỉ lệ rớt do cháy môn >= 0.04,Tỉ lệ rớt do cháy môn >= 0.06,Tỉ lệ rớt do cháy môn >= 0.08,...,Số lượng sv mỗi lớp >= 100,Số lượng sv mỗi lớp >= 200,Số lượng sv mỗi lớp >= 300,Số lượng sv mỗi lớp >= 400,Số lượng sv mỗi lớp >= 500,Số lượng sv học lại >= 10,Số lượng sv học lại >= 20,Số lượng sv học lại >= 30,Số lượng sv học lại >= 40,Số lượng sv học lại >= 50
0,ACC101,0,1,0,0,0,1,0,0,0,...,1,1,0,0,0,1,0,0,0,0
1,ADY201m,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,AET102,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AFA201,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AID301c,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,ÐBA102,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
213,ÐNG102,1,0,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
214,ÐSA102,0,0,1,0,0,1,1,0,0,...,1,1,1,1,1,1,0,0,0,0
215,ÐTB102,0,0,1,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0


# Tạo DataFrame theo tính chất môn

In [ ]:
#  Imports & Loading Data ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình hiển thị biểu đồ đẹp hơn
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Đọc dữ liệu (Thay đổi đường dẫn file nếu cần)
file_path = "/content/drive/MyDrive/Dự án/SubjectCode.xlsx"
try:
    df2 = pd.read_excel(file_path, sheet_name="KQ")
    print("Đọc dữ liệu thành công!")
    display(df2.head()) # Hiển thị 5 dòng đầu
    print(df2.info())   # Kiểm tra kiểu dữ liệu
except FileNotFoundError:
    print(f"Không tìm thấy file tại: {file_path}. Vui lòng kiểm tra lại đường dẫn.")

Đọc dữ liệu thành công!


,Môn,Phân loại,Note
0,PRO192,Code,Lập trình hướng đối tượng
1,JPD123,Ngôn ngữ,Tiếng Nhật
2,MAD101,Logic,Toán rời rạc
3,ITE302c,Môn cơ bản,Đạo đức trong CNTT
4,ÐSA102,Năng khiếu,Sáo


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Môn        217 non-null    object
 1   Phân loại  214 non-null    object
 2   Note       198 non-null    object
dtypes: object(3)
memory usage: 5.2+ KB
None


In [ ]:
df2 = df2.drop(columns = 'Note',axis=1)
df2.rename(columns={"Môn": "SubjectCode"}, inplace=True)
df2

,SubjectCode,Phân loại
0,PRO192,Code
1,JPD123,Ngôn ngữ
2,MAD101,Logic
3,ITE302c,Môn cơ bản
4,ÐSA102,Năng khiếu
...,...,...
212,MSM201c,Môn cơ bản
213,MCO201m,Môn chuyên ngành
214,ENM301,Ngôn ngữ
215,ECO111,Môn chuyên ngành


# Tạo DataFrame encode one-hot thứ hai

In [ ]:
import pandas as pd

# 1. Lấy Subjectcode
subject_code = df2["SubjectCode"]

# 2. One-hot 0–1 cho cột "Phân loại"
df2_onehot = pd.get_dummies(
    df2["Phân loại"],
    prefix="Môn",
    dtype=int
)

# 3. Ghép Subjectcode vào DF one-hot
df2_onehot = pd.concat(
    [subject_code, df2_onehot],
    axis=1
)
df2_onehot

,SubjectCode,Môn_Code,Môn_Kĩ năng mềm,Môn_Logic,Môn_Môn chuyên ngành,Môn_Môn cơ bản,Môn_Ngôn ngữ,Môn_Năng khiếu
0,PRO192,1,0,0,0,0,0,0
1,JPD123,0,0,0,0,0,1,0
2,MAD101,0,0,1,0,0,0,0
3,ITE302c,0,0,0,0,1,0,0
4,ÐSA102,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
212,MSM201c,0,0,0,0,1,0,0
213,MCO201m,0,0,0,1,0,0,0
214,ENM301,0,0,0,0,0,1,0
215,ECO111,0,0,0,1,0,0,0


# DataFrame Feature tổng

In [ ]:
df_mon = pd.merge(course_features, df2, on="SubjectCode", how="inner")
df_mon

,SubjectCode,Tỉ lệ rớt do cháy môn,Tỉ lệ rớt do không đủ điểm,Giỏi,Khá,Khá Giỏi,Rớt môn,Xuất Sắc,Mức độ dễ lấy điểm,Số lượng lớp,Số lượng sv mỗi lớp,Số lượng sv học lại,Phân loại
0,ACC101,0.039604,0.039604,0.153465,0.455446,0.297030,0.074257,0.019802,1.589109,39,202,12,Môn chuyên ngành
1,ADY201m,0.011299,0.016949,0.514124,0.107345,0.271186,0.016949,0.090395,2.553672,15,177,5,Code
2,AET102,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1,1,0,Môn cơ bản
3,AFA201,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1,1,0,Môn chuyên ngành
4,AID301c,0.000000,0.000000,0.311111,0.200000,0.455556,0.000000,0.033333,2.177778,2,90,0,Môn chuyên ngành
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,ÐBA102,0.022901,0.000000,0.419847,0.091603,0.251908,0.022901,0.213740,2.709924,25,131,2,Năng khiếu
213,ÐNG102,0.033333,0.009524,0.385714,0.161905,0.290476,0.042857,0.119048,2.376190,33,210,3,Năng khiếu
214,ÐSA102,0.050000,0.011429,0.308571,0.157143,0.321429,0.061429,0.151429,2.331429,83,700,11,Năng khiếu
215,ÐTB102,0.052174,0.000000,0.269565,0.182609,0.295652,0.052174,0.200000,2.382609,20,115,2,Năng khiếu


# DataFrame one-hot tổng

In [ ]:
df_mon_onehot = pd.merge(course_features_onehot, df2_onehot, on="SubjectCode", how="inner")
df_mon_onehot

,SubjectCode,KQ__Giỏi,KQ__Khá,KQ__Khá Giỏi,KQ__Rớt môn,KQ__Xuất Sắc,Tỉ lệ rớt do cháy môn >= 0.02,Tỉ lệ rớt do cháy môn >= 0.04,Tỉ lệ rớt do cháy môn >= 0.06,Tỉ lệ rớt do cháy môn >= 0.08,...,Số lượng sv học lại >= 30,Số lượng sv học lại >= 40,Số lượng sv học lại >= 50,Môn_Code,Môn_Kĩ năng mềm,Môn_Logic,Môn_Môn chuyên ngành,Môn_Môn cơ bản,Môn_Ngôn ngữ,Môn_Năng khiếu
0,ACC101,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,ADY201m,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,AET102,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,AFA201,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,AID301c,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,ÐBA102,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
213,ÐNG102,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
214,ÐSA102,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
215,ÐTB102,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
